In [1]:
#importamos algoritmos
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.algorithms.moo.sms import SMSEMOA
#importamos indicadores
from pymoo.indicators.igd_plus import IGDPlus
from pymoo.indicators.hv import HV
#importamos función para resolver problemas de optimización
from pymoo.optimize import minimize
#importamos funciones para visualizar resultados
from pymoo.util.display.column import Column
from pymoo.util.display.output import Output
#usamos pandas para nuestro dataframe
import pandas as pd
#numpy para algunos cálclulos
import numpy as np
#importamos función para traer problemas
from pymoo.problems import get_problem
#función para ref_dirs
from pymoo.util.ref_dirs import get_reference_directions
#función para detener algortimo
from pymoo.termination import get_termination
#time para generar la primer semilla
import time

In [2]:
class Salida(Output):
    def __init__(self):
        super().__init__()
        #definimos columnas
        self.f_igdp = Column("igd+", width=13)
        self.f_hv = Column("hv", width=13)
        self.time = Column("time", width=13)
        self.x=Column("X",width=13)
        self.f=Column("F",width=13)
        #añadimos las columnas a las ya existentes
        self.columns += [self.f_hv,self.f_igdp,self.time,self.x,self.f]

    def update(self, algorithm):
        super().update(algorithm)
        #usamos igdp con el frente de pareto
        pf=algorithm.problem.pareto_front()
        ind = IGDPlus(pf)
        self.f_igdp.set(ind(algorithm.opt.get("F")))
        #punto que utilizaremos para hv
        ref_point = np.array([3,5,7])
        ind2 = HV(ref_point=ref_point)
        self.f_hv.set(ind2(algorithm.opt.get("F")))
        #tiempo de ejecución
        self.time.set(algorithm.result().exec_time)
        #resultados
        self.f.set(algorithm.opt.get("F"))
        self.x.set(algorithm.opt.get('X'))

In [ ]:
a

In [ ]:
a
hora=int(time.time())
hora

In [3]:
hora=1666209266

In [4]:
problemas=["wfg1","wfg2","wfg3","wfg4","wfg5","wfg6","wfg7","wfg8","wfg9"]
num_total=20*len(problemas)
np.random.seed(hora)
semillas=np.random.randint(1,10000,num_total+1)

In [5]:
ref_dirs=get_reference_directions("energy", 3, 200, seed=semillas[-1])
algoritmos={'NSGA2':NSGA2(pop_size=200,prob_neighbor_mating=0.8,n_neighbors=10),
            'NSGA3':NSGA3(pop_size=200,ref_dirs=ref_dirs,prob_neighbor_mating=0.8,n_neighbors=10),
            'MOEAD':MOEAD(ref_dirs=ref_dirs,prob_neighbor_mating=0.8,n_neighbors=10)}
            #,'SMSEMOA':SMSEMOA(pop_size=200,ref_dirs=ref_dirs,prob_neighbor_mating=0.8,n_neighbors=10)}
termination = get_termination("n_gen", 500)

In [6]:
df=pd.DataFrame(columns=["Epoca","Fe","hv","igdp","Tiempo","X","F","Ejecucion","Algoritmo","Problema"])
df.to_csv('resultados.csv',index=False,mode='a')

In [ ]:
lista=[]
j=0
for problema in problemas:
    for algoritmo in algoritmos:
        #k se reinicia en cada algoritmo nuevo para que tengan la misma semilla diferentes algoritmos
        k=0
        for i in range(20):
            print(algoritmo)
            print(problema)
            print(i)
            semilla=semillas[j+k]
            print(j+k)
            lista.append([algoritmo,problema,i,semilla,j])
            algorithm = algoritmos[algoritmo]
            problem=get_problem(problema,n_var=12,n_obj=3)
            result=minimize(
                problem=problem,algorithm=algorithm,termination=termination,
                verbose=True,output=Salida(),save_history=True,seed=semilla
            )
            lista=[[columna.value for columna in observacion.output.columns] for observacion in result.history ]
            temporal=pd.DataFrame(lista,columns=["Epoca","Fe","hv","igdp","Tiempo","X","F"])
            temporal['Ejecucion']=i+1
            temporal['Algoritmo']=algoritmo
            temporal['Problema']=problema
            temporal.to_csv('resultados1.csv',index=False,mode='a',header=False,
                            columns=["Epoca","Fe","hv","igdp","Tiempo","X","F","Ejecucion","Algoritmo","Problema"])
            x=pd.DataFrame(result.algorithm.opt.get('X'))
            x.to_csv('x.csv',index=False,mode='a')
            metadatos=pd.DataFrame([[algoritmo,problema,i+1]])
            metadatos.to_csv('metadatos.csv',index=False,mode='a')
            k+=1
    j+=k


NSGA2
wfg1
0
0
n_gen  |  n_eval  |       hv      |      igd+     |      time     |       X       |       F      
     1 |      200 |  7.9199527784 |  2.1908949325 |  0.5536012650 | [[1.68401217e | [[2.8655609  
     2 |      400 |  8.4865024966 |  2.1694280393 |  0.6199662685 | [[ 1.68401217 | [[2.8655609  
     3 |      600 |  9.0462669094 |  2.1509161882 |  0.6867973804 | [[8.58777299e | [[2.7876751  
     4 |      800 |  1.158288E+01 |  2.0535722936 |  0.7342417240 | [[8.58777299e | [[2.7876751  
     5 |     1000 |  1.192397E+01 |  2.0439282207 |  0.8291406631 | [[8.58777299e | [[2.7876751  
     6 |     1200 |  1.386292E+01 |  1.9988580083 |  0.8909590244 | [[8.58777299e | [[2.7876751  
     7 |     1400 |  1.395609E+01 |  1.9946451631 |  0.9428491592 | [[8.58777299e | [[2.7876751  
     8 |     1600 |  1.550949E+01 |  1.9413996912 |  0.9949274063 | [[8.58777299e | [[2.7876751  
     9 |     1800 |  1.788982E+01 |  1.8909218353 |  1.0603086948 | [[8.58777299e | [[2.7876751  
    1

    84 |    16800 |  3.837082E+01 |  1.2530846972 |  6.6109738350 | [[3.25715362e | [[1.56297024 
    85 |    17000 |  3.892704E+01 |  1.2478722818 |  6.6819531918 | [[3.38057920e | [[1.55731826 
    86 |    17200 |  3.894144E+01 |  1.2415320840 |  6.7514514923 | [[1.36492861e | [[2.57484825 
    87 |    17400 |  3.898999E+01 |  1.2371566974 |  6.8234946728 | [[3.38057920e | [[1.55731826 
    88 |    17600 |  3.932147E+01 |  1.2346925138 |  6.8968465328 | [[3.38057920e | [[1.55731826 
    89 |    17800 |  3.942085E+01 |  1.2294830645 |  6.9584507942 | [[1.36492861e | [[2.57484825 
    90 |    18000 |  3.976889E+01 |  1.2261385973 |  7.0199360847 | [[2.76299991e | [[1.53849461 
